In [1]:
# activate crawler env 1st!
import netCDF4 as nc
from netCDF4 import Dataset

In [2]:
#The original problem was in Kyle's program where the thredds crawler got OPENDAP URLs
# and seemed to point to using ncattrs syntax as the problem
# trying to duplicate without having to run the crawler to get the URL
#
# this runs, but says none of the required_attributes are there
# when the file HAS MOORING and INST_TYPE, but not serial_number

required_attributes = ('MOORING', 'INST_TYPE', 'serial_number')
#fn='http://geoport.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data_tests/V_fv/SANDWICH2016/10631whVp-cal.nc'
fn='http://geoport.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data/SANDWICH2016/10631whVp-cal.nc'
try:
    with nc.Dataset(fn) as fnc:
        for k in required_attributes:
            # .ncattrs returns a list of global attributes 
            if k not in fnc.ncattrs():
                msg = k + ' not found in ' + fn
                print(msg)
            else:
                print('found ',k)                
except OSError:
    print('failed trying to read ' + spec_file)


MOORING not found in http://geoport.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data/SANDWICH2016/10631whVp-cal.nc
INST_TYPE not found in http://geoport.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data/SANDWICH2016/10631whVp-cal.nc
serial_number not found in http://geoport.whoi.edu/thredds/dodsC/silt/usgs/Projects/stellwagen/Data/SANDWICH2016/10631whVp-cal.nc


In [3]:
# I downloaded the file from SANDWICH2016 to try to find out what messes up netcdf4
# when I try to interact with the attributes
badnc=nc.Dataset('/home/ellyn/python_progs/conversions/10631whVp-cal.nc')

In [11]:
print(badnc.ncattrs())

['outFileRoot', 'MOORING', 'Deployment_date', 'Recovery_date', 'DATA_ORIGIN', 'WATER_DEPTH', 'WATER_DEPTH_datum', 'EXPERIMENT', 'PROJECT', 'DESCRIPT', 'DATA_CMNT', 'SciPi', 'latitudes', 'longitude', 'magnetic_variation_at_site', 'magnetic_variation_applied', 'ADCP_serial_number', 'ADCPBinSize', 'wavesmon_version', 'wavesmon_isV', 'metafile_name', 'metafile_author', 'metafile_version', 'logfilename', 'CREATION_DATE', 'INST_TYPE', 'WATER_DEPTH_source', 'DATA_TYPE', 'DATA_SUBTYPE', 'COORD_SYSTEM', 'WATER_MASS', 'POS_CONST', 'DEPTH_CONST', 'DRIFTER', 'VAR_FILL', 'FILL_FLAG', 'COMPOSITE', 'platform_type', 'history', 'latitude', 'latitude_units', 'longitude_units', 'modification_date', 'initial_instrument_height', 'initial_instrument_height_note', 'nominal_sensor_depth', 'nominal_sensor_depth_note', 'WavesMonCfg.FileOK', 'WavesMonCfg.ErrorLogging', 'WavesMonCfg.Auto_Configure', 'WavesMonCfg.InPath', 'WavesMonCfg.InFile', 'WavesMonCfg.Velocity_Text_File', 'WavesMonCfg.OutPath', 'WavesMonCfg.O

In [4]:
badnc.DATA_TYPE

'WAVESPEC'

In [5]:
# I think the problem is the '.' that's in the attribute name, when used in this way 
# Python3 seems to interpret the .NBeams as a method that it can't find.
# all the WavesMon* global attributes can't be read,
badnc.WavesMonCfg.NBeams

AttributeError: NetCDF: Attribute not found

In [6]:
#So here's the equivalent on the local code
k='WavesMonCfg.NBeams'
if k in badnc.ncattrs():
    msg = k + ' found in global atts'
    print(msg)

WavesMonCfg.NBeams found in global atts


In [7]:
# seems like you can access it from a dict though
all_atts=badnc.__dict__
all_atts['WavesMonCfg.NBeams']

4.0

In [8]:
# other aspects of the file seem to work fine
# I can query the dimensions
badnc.dimensions

OrderedDict([('time',
              <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 2393),
             ('lat',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 1),
             ('lon',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1),
             ('frequency',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'frequency', size = 64),
             ('direction',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'direction', size = 360)])

In [9]:
#Access variables
wvht = badnc.variables.get("wh_4061")
whatts=wvht.__dict__
if 'units' in whatts:
    print('wave height units are : ', getattr(wvht,'units'))

wave height units are :  m
